In [1]:
# This Python script demonstrates how to use the SPARQL language to query a Brick model.
# SPARQL: A query language for retrieving and manipulating data stored in Resource Description Framework (RDF) format

# Install necessary Python pakcages
import pandas as pd
from rdflib.namespace import RDFS, SKOS
from pandas import DataFrame
from rdflib import Graph, URIRef
import brickschema
from brickschema.namespaces import A, BRICK, UNIT, XSD
import datetime

-----------

#### **Accessing brick model in turtle (.ttl) file**

In [2]:
file = 'PV generation system metadata.ttl'
graph = Graph()
graph.parse(file, format='turtle')

<Graph identifier=Nb459335c3e7945f3b3a92fa92226059f (<class 'rdflib.graph.Graph'>)>

----------

#### **Inquiring overall information of power stations**

##### 1.Number of PV stations

In [3]:
q = """
select (count(*) as ?tripleCount) where {
  ?station a brick:PV_Generation_System.
}
"""
res = graph.query(q)
for row in res:
    count = row[0]
    print(f"There are {count} PV stations on campus.")

There are 60 PV stations on campus.


##### 2.Types of PV inverter

In [4]:
q = """
select (count(distinct ?inverterModel) as ?numInverterModels) where {
  ?inverter a brick:Inverter;
           ext:module [ brick:value ?inverterModel] 
}
"""
res = graph.query(q)
for row in res:
    count = row[0]
    print(f"There are {count} types of PV inverter on campus.")

There are 23 types of PV inverter on campus.


##### 3.Types of PV module

In [5]:
q = """
select (count(distinct ?moduleModel) as ?numModuleModels) where {
  ?module a brick:PV_Panel;
           ext:module [ brick:value ?moduleModel] 
}
"""
res = graph.query(q)
for row in res:
    count = row[0]
    print(f"There are {count} types of PV module on campus.")

There are 8 types of PV module on campus.


##### 4.Campus PV station capacity (Sum, Min, Max, Avg)

In [6]:
q1 = """
select (SUM(?capacity) as ?sumcapacity) where {
   ?station a brick:PV_Generation_System;
           ext:ratedPowerOutput [ brick:hasUnit unit:KW; brick:value ?capacity];
}
"""
res = graph.query(q1)
for row in res:
    total = row[0]
    print(f"Total capacity of PV stations is {total} kW.")

q2 = """
select (MIN(?capacity) as ?mincapacity) where {
   ?station a brick:PV_Generation_System;
           ext:ratedPowerOutput [ brick:hasUnit unit:KW; brick:value ?capacity];
}
"""
res = graph.query(q2)
for row in res:
    min = row[0]
    print(f"The minimum capacity of single PV station is {min} kW..")


q3 = """
select (MAX(?capacity) as ?maxcapacity) where {
   ?station a brick:PV_Generation_System;
           ext:ratedPowerOutput [ brick:hasUnit unit:KW; brick:value ?capacity];
}
"""
res = graph.query(q3)
for row in res:
    max = row[0]
    print(f"The maximum capacity of single PV station is {max} kW.")
    
q4 = """
select (AVG(?capacity)  as ?avgcapacity) where {
   ?station a brick:PV_Generation_System;
           ext:ratedPowerOutput [ brick:hasUnit unit:KW; brick:value ?capacity];}
"""
res = graph.query(q4)
for row in res:
    mean = row[0]
    print(f"The mean capacity of single PV station is {mean} kW.")

Total capacity of PV stations is 2230.81 kW.
The minimum capacity of single PV station is 3.72 kW..
The maximum capacity of single PV station is 249.5 kW.
The mean capacity of single PV station is 37.18016666666666666666666667 kW.


----------------

#### **Time related inqueries**

##### 1.The earliest and latest constructed PV station

In [7]:
q1 = """
select (MIN(?Connectiontime) as ?earliesttime) where {
    ?station a brick:PV_Generation_System;
           ext:connectionDate [  brick:value ?Connectiontime];
}
"""
q2 = """
select (MAX(?Connectiontime) as ?latesttime) where {
    ?station a brick:PV_Generation_System;
           ext:connectionDate [  brick:value ?Connectiontime];
}
"""

res = graph.query(q1)
for row in res:
    earliestdate = row[0]
    print(f"The earliest PV station was constructed on {earliestdate} .")
res = graph.query(q2)
for row in res:
    latesetdate = row[0]
    print(f"The lateset PV station was constructed on {latesetdate} .")

The earliest PV station was constructed on 2020-12-01T00:00:00 .
The lateset PV station was constructed on 2023-07-29T00:00:00 .


##### 2.Operation period of each station

In [8]:
q = """
select ?station ?Connectiontime  where {
    ?station a brick:PV_Generation_System;
           ext:connectionDate [  brick:value ?Connectiontime];
}
"""
res = graph.query(q)

today=datetime.date.today()
today=pd.to_datetime(today)

time_df=DataFrame(res, columns=res.vars)
time_df.columns = time_df.columns.str.strip()
time_df[['Namespace', 'Sitename']] = time_df["station"].str.split('#', expand=True)
#time_df=time_df.drop(['Site'], axis=1)
time_df=time_df[["Sitename","Connectiontime"]]
time_df["Connectiontime"]=pd.to_datetime(time_df["Connectiontime"])
time_df["Operation Days"]=(today-(time_df["Connectiontime"])).dt.days
time_df["Operation Years"]=(time_df["Operation Days"]/365).round(2)

time_df

,Sitename,Connectiontime,Operation Days,Operation Years
0,CYT__Station_1_,2021-01-30,1130,3.10
1,CYT__Station_2_,2021-01-30,1130,3.10
2,Indoor_Sport_Center,2021-12-01,825,2.26
3,LSK_North,2021-04-30,1040,2.85
4,LSK_South,2021-04-30,1040,2.85
5,Library,2022-05-31,644,1.76
6,SQ1,2021-04-30,1040,2.85
7,SQ10,2021-04-30,1040,2.85
8,SQ11,2021-04-30,1040,2.85
9,SQ12,2021-02-27,1102,3.02


-------

#### **Specific Site Information** （Taking the Indoor Sport Center site as an example）

##### 1.Site Information

In [9]:
q = """
select ?Latitude_degree ?Longitude_degree ?Altitude_m ?Ratedpower_kW ?Connectiontime ?Contractor where {
  pvsystem:Indoor_Sport_Center a brick:PV_Generation_System ;
    brick:coordinates [ brick:latitude ?Latitude_degree ;
            brick:longitude ?Longitude_degree ] ;
    ext:altitude [ brick:hasUnit unit:M ;
            brick:value ?Altitude_m ] ;
    ext:connectionDate [ brick:value ?Connectiontime ] ;
    ext:contractor [ brick:value ?Contractor ] ;
    ext:ratedPowerOutput [ brick:hasUnit unit:KW ;
            brick:value ?Ratedpower_kW ] .
}
"""
res = graph.query(q)

site_information=DataFrame(res, columns=res.vars)
site_information

,Latitude_degree,Longitude_degree,Altitude_m,Ratedpower_kW,Connectiontime,Contractor
0,22.28,114.17,3.0,33.3,2021-12-01T00:00:00,Indoor_Sport_Centre_Building


##### 2.Inverter Information

In [10]:
q = """
select ?Brand ?Module ?Number where {
  pvsystem:Indoor_Sport_Center_Inverter a brick:Inverter  ;
    ext:brand [ brick:value ?Brand ] ;
    ext:module [ brick:value ?Module ] ;
    ext:number [ brick:value ?Number ].
}
"""
res = graph.query(q)

inverter_information=DataFrame(res, columns=res.vars)
inverter_information

,Brand,Module,Number
0,SMA,Sunny_Tripower_STP_10_0,1


##### 3.Module Information

In [11]:
q = """
select ?Brand ?Module ?RatedPower_W ?Number ?PanelGegradationRate_FirstYear_percent ?PanelLinearGegradation_percent ?Type  where {
pvsystem:Indoor_Sport_Center_Module a brick:PV_Panel ;
    ext:brand [ brick:value ?Brand ] ;
    ext:module [ brick:value ?Module ] ;
    ext:number [ brick:value ?Number ] ;
    ext:panelDegradationFirstYear [ brick:hasUnit unit:PERCENT ;
            brick:value ?PanelGegradationRate_FirstYear_percent ] ;
    ext:panelLinearDegradation [ brick:hasUnit unit:PERCENT ;
            brick:value ?PanelLinearGegradation_percent] ;
    ext:ratedPowerOutput [ brick:hasUnit unit:W ;
            brick:value ?RatedPower_W ] ;
    ext:specificType [ brick:value ?Type ] .
}
"""
res = graph.query(q)

module_information=DataFrame(res, columns=res.vars)
module_information

,Brand,Module,RatedPower_W,Number,PanelGegradationRate_FirstYear_percent,PanelLinearGegradation_percent,Type
0,JinkoSolar,JKM365N_6TL3,365.0,120,1.0,0.4,Si_mono_Model
